In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sympy import symbols, nsolve
from sympy import cosh, sin , sinh, cos,exp
from scipy.optimize import fsolve
from scipy.integrate import quad

# System constants
e2, $\hbar$ ,amu, Q, parameter P

In [2]:
e2=1.43997 ; hbarc=197.3269718 ; amu=931.4943
z1=82 ; m1=207
z2=2 ; m2=4
Q=7.599
mu=m1*m2/(m1+m2)*amu

# Para's values of pot  
$v_0,a_0,r_0$  
$r_c=r_0$

In [3]:
v0=162.3 ; a0=0.4 ; r0=7.107 ; l=5 ; P=0.3

# Physical model section
$V_N$: cosh           
$V_C$: piecewise    
$V_{pot}$: $V_N+V_C+(l+\frac{1}{2})^2/r^2$  
$k(r)=\sqrt{\frac{2\mu}{\hbar^2}|Q-V(r)|}$  
$F\int_{r_1}^{r_2}dr \frac{1}{k(r)}cos^2[\int_{r_1}^rdr'k(r')-\frac{\pi}{4}]=1$  
F=$1/\int_{r_1}^{r_2}dr \frac{1}{k(r)}cos^2[\int_{r_1}^rdr'k(r')-\frac{\pi}{4}]$
Def wave number k and $\Gamma=PF\frac{\hbar^2}{4\mu}exp[-2\int_{r_2}^{r_3} k(r)dr]$   
$T_{1/2}=\hbar ln2/\Gamma$

In [4]:
def wspot(r,v0,a0,r0):
    return -v0*(1+np.cosh(r0/a0))/(np.cosh(r/a0)+np.cosh(r0/a0))

def vc(r,z1,z2,rc):
    return np.where(
        r < rc,
        z1 * z2 * e2 * (3 - r**2 / rc**2) /(2 * rc),
        z1 * z2 * e2 / r
    )
    
def vpot(r,v0,a0,r0,z1,z2,rc,l):
    return wspot(r,v0,a0,r0)+vc(r,z1,z2,rc)+(l+0.5)**2/r**2*hbarc**2/(2*mu)

In [5]:
def Model(m1,m2,z1,z2,l,Q,v0,a0,r0,P):
    rc=r0

    def f(r):
        return vpot(r, v0, a0, r0, z1, z2, rc, l)-Q

    x_values = np.linspace(0.1, 80, 100)
    f_values = f(x_values)
    roots = []

    for i in range(len(f_values) - 1):
        # Check for a sign change
        if np.sign(f_values[i]) != np.sign(f_values[i + 1]):
            # Estimate root using fsolve, starting from the midpoint
            guess = (x_values[i] + x_values[i + 1]) / 2
            root = fsolve(f, guess)[0]
            # Avoid duplicate roots
            if root > 0 and not any(np.isclose(root, r, atol=1e-5) for r in roots):
                roots.append(root)

    roots.sort()        # sort from small roots to large roots


    def k(r):
        return np.sqrt(2*mu/hbarc**2*np.abs(Q-vpot(r,v0,a0,r0,z1,z2,rc,l))) 
    #内部积分函数
    def inner_integral(r):
        result, _ = quad(k, roots[0], r)
        return result
    #被积函数
    def integrand(r):
        inner_int = inner_integral(r)
        return (1 / k(r)) * np.cos(inner_int - np.pi / 4)**2

    integral_result, _ = quad(integrand, roots[0], roots[1])    
    F=1/integral_result

    result,_=quad(k,roots[1],roots[2])
    # print("result:",result)

    gamma=P*F*hbarc**2/(4*mu)*exp(-2*result)
    print("gamma:",gamma,"MeV")

    T_half=hbarc*np.log(2)/gamma
    print("T_half:",T_half,"fm")

    T_half=T_half*1e-23/3
    print('T_half',T_half,'s')

    return T_half

In [ ]:
test=Model(m1,m2,z1,z2,l,Q,v0,a0,r0,P)
# print(test)

# emcee to mcmc the log posterior
log_posterior function:  
y: data  $\to$ lifetime $T_{1/2}$  
sigma: data point uncertainty  
a: para vector

In [7]:
y=np.array([0.516]) #实验数据: 207Pb
sigma=np.array([0.01])

def log_prior(a):
    vv,aa,rr,pp=a
    R=0.5
    sigma_v=2*R
    sigma_a=0.1*R
    sigma_r=0.5*R
    sigma_P=0.1*R

    prior_v=-0.5*(vv-v0)**2/sigma_v**2
    prior_a=-0.5*(aa-a0)**2/sigma_a**2
    prior_r=-0.5*(rr-r0)**2/sigma_r**2
    prior_P=-0.5*(pp-P0)**2/sigma_P**2

    return prior_v+prior_a+prior_r+prior_P

def log_posterior(a,y,sigma,m1,m2,z1,z2,l,Q):

    prior_value=log_prior(a)   
    f=Model(m1,m2,z1,z2,l,Q,a[0],a[1],a[2],a[3])

    return -0.5*(y-f)**2/sigma**2+log_prior(a)

M: number of paras for BA

In [ ]:
M=4 
nwalkers=2*M
initial_pos = [v0, a0, r0, P0]
perturbation_scale = [0.1, 0.01, 0.1, 0.01]  # 每个参数的扰动大小

a = np.array([initial_pos + np.random.normal(0, perturbation_scale, M) for _ in range(nwalkers)])

print(a)

In [ ]:
import emcee
sampler = emcee.EnsembleSampler(nwalkers, M, log_posterior, args=[y,sigma,m1,m2,z1,z2,l,Q],a=0.2,pool=pool)
state = sampler.run_mcmc(a, 200)
sampler.reset()
sampler.run_mcmc(state,2000)

# corner to give posterior figures

In [ ]:
import prettyplease
samples = sampler.get_chain(flat=True)
# plt.hist(samples[:,3], 100, color="k", histtype="step", density=True)
# plt.xlabel(r"$\theta$")
# plt.ylabel(r"$p(\theta|D)$")
# plt.gca().set_yticks([]);
labels = ["$v_0$", "$a_0$", "$r_0$", "$P_0$"]
fig = prettyplease.corner(samples,labels=labels)
plt.show()  